<a href="https://colab.research.google.com/github/GomideB/Agente-AI-Primeiros-Socorros/blob/main/Primeiros_Socorros_AGENTE_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
# Instalação de bibliotecas essenciais para o projeto.
# - 'google-genai': Para interagir com os modelos Gemini (Google AI Studio).
# - 'google-adk': Google Agent Development Kit, que facilita a construção de agentes de IA.
# - 'aiohttp': Para fazer requisições HTTP assíncronas, crucial para as chamadas à API do Google Maps.
%pip -q install google-genai google-adk aiohttp

In [73]:
# --- Configurações Iniciais e Importações ---
import os
import asyncio
import aiohttp
import google.genai
from google.colab import userdata
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from typing import List, Dict, Optional
import nest_asyncio
import textwrap
from IPython.display import display, Markdown, HTML

# --- INÍCIO DA SUPRESSÃO DE LOGS INDESEJADOS ---
# Suprime avisos e erros específicos de bibliotecas que podem ser ruidosos no console,
# comuns em ambientes de notebook com código assíncrono.
import logging
import warnings

# Suprime avisos e erros do logger 'asyncio' (e.g., 'Task was destroyed but it is pending!')
logging.getLogger('asyncio').setLevel(logging.CRITICAL)

# Suprime avisos e erros do logger 'httpx' (usado internamente por google-genai/adk para requisições HTTP)
logging.getLogger('httpx').setLevel(logging.CRITICAL)

# Suprime outros avisos gerais do Python (e.g., de bibliotecas)
warnings.filterwarnings("ignore")
# --- FIM DA SUPRESSÃO DE LOGS INDESEJADOS ---

# 'nest_asyncio.apply()' permite aninhar loops de evento asyncio,
# o que é necessário para rodar código assínrono em ambientes como o Google Colab,
# onde um loop de evento já pode estar ativo.
nest_asyncio.apply()

In [74]:
# --- Configuração das Chaves de API ---
# 'userdata.get()' é uma função do Colab para acessar segredos de forma segura.
# Certifique-se de que 'GOOGLE_API_KEY' (para Gemini) e 'Maps_API' (para Google Maps)
# estão configuradas nas "Secrets" do Google Colab.
os.environ["GOOGLE_API_KEY2"] = userdata.get('GOOGLE_API_KEY2')
os.environ["GOOGLE_MAPS_API"] = userdata.get('GOOGLE_MAPS_API')
# A chave da API do Google Maps é armazenada em uma variável local para uso nas funções.
MAPS_API_KEY = os.environ["GOOGLE_MAPS_API"]

# Inicialização do cliente Gemini.
# Este cliente deve ser inicializado uma única vez no início do script para gerenciar as chamadas à API Gemini.
client = google.genai.Client()

# --- Constantes do Projeto ---
# Define um raio de busca padrão em quilômetros para as APIs de localização.
DEFAULT_SEARCH_RADIUS_KM = 50

In [75]:
# --- Funções Auxiliares Comuns ---
def executar_agente(agent: Agent, mensagem: str) -> str:
    """
    Executa um agente de IA do Google ADK com uma mensagem específica e coleta sua resposta final.
    Esta função encapsula a lógica de criação de sessão e execução do agente,
    garantindo que o fluxo de interação seja consistente para todos os agentes.

    Args:
        agent (Agent): A instância do agente ADK (ex: agente_triagem, agente_planejamento_acoes).
        mensagem (str): A mensagem de entrada (prompt) a ser enviada para o agente.

    Returns:
        str: A resposta final gerada pelo agente, sem espaços em branco no início/fim.
    """
    # Cria um serviço de sessão em memória. Ele gerencia o estado da conversa com o agente.
    session_service = InMemorySessionService()
    # Cria uma sessão específica para um usuário ('user1') e um aplicativo (o nome do agente).
    # Isso permite que o agente mantenha o contexto através de múltiplas interações.
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

    # Cria um 'Runner' que é responsável por orquestrar a execução do agente.
    # O Runner utiliza o agente e o serviço de sessão para processar as mensagens.
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    # Converte a mensagem de entrada do usuário para o formato de conteúdo esperado pelo agente.
    content = types.Content(role="user", parts=[types.Part(text=mensagem)])
    resposta = ""

    # Executa o agente e itera sobre os eventos retornados.
    # O método 'runner.run()' retorna um gerador de eventos.
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        # Verifica se o evento atual contém a resposta final do agente.
        if event.is_final_response():
            # Concatena todas as partes de texto da resposta final.
            for part in event.content.parts:
                if part.text:
                    resposta += part.text + "\n"
    return resposta.strip() # Retorna a resposta final, removendo quebras de linha/espaços extras.

def to_markdown(text: str) -> Markdown:
    """
    Formata uma string para exibição como Markdown no Google Colab,
    adicionando uma indentação visual de citação e padronizando bullet points.

    Args:
        text (str): O texto a ser formatado.

    Returns:
        Markdown: Um objeto Markdown que pode ser exibido no Colab.
    """
    # Substitui bullet points unicode por asteriscos para renderização consistente em Markdown.
    text = text.replace('•', ' *')
    # Adiciona uma indentação tipo "bloco de citação" (>) a cada linha do texto.
    return Markdown(textwrap.indent(text, '> '))

In [76]:
# --- Definição e Instanciação Única dos Agentes ---
# Cada agente é uma instância da classe 'Agent' do ADK, configurada com um nome,
# modelo de linguagem (Gemini), instruções específicas para sua tarefa e uma descrição.
# Alguns agentes podem ter ferramentas associadas, como a 'Google Search'.

# Agente 1: Agente de Triagem Inicial
agente_triagem = Agent(
    name="agente_triagem",
    model="gemini-1.5-flash", # Modelo Gemini otimizado para velocidade.
    instruction="""Você é um assistente de primeiros socorros.
    Pergunte ao usuário sobre o que ele está sentindo para entender a situação de emergência.
    Mantenha a conversa breve e foque em obter uma descrição clara dos sintomas.
    Após a descrição inicial, **pergunte:** "Você sente mais alguma dor ou algo diferente que eu deva saber?"
    """,
    description="Agente para coletar os sintomas iniciais do usuário."
)

In [77]:
# Agente 2: Agente de Coleta e Consolidação de Sintomas Adicionais
agente_coleta_sintomas_adicionais = Agent(
    name="agente_coleta_sintomas_adicionais",
    model="gemini-1.5-flash",
    instruction="""Você é um assistente de saúde.
    Sua tarefa é consolidar informações de sintomas. Receberá um resumo dos sintomas iniciais
    e uma descrição de sintomas adicionais fornecidos pelo usuário.
    Crie uma descrição única, clara e completa de TODOS os sintomas do paciente,
    sem incluir nenhuma pergunta ou instrução adicional. Se o usuário indicou que não há
    sintomas adicionais, apenas forneça o resumo dos sintomas iniciais.
    """,
    description="Agente para coletar e consolidar sintomas adicionais do usuário."
)

In [78]:
# Agente 3: Agente de Busca de Primeiros Socorros
agente_busca_primeiros_socorros = Agent(
    name="agente_busca_primeiros_socorros",
    model="gemini-1.5-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Utilize a ferramenta de busca do Google para encontrar informações confiáveis e atualizadas
    sobre como tratar a condição médica descrita. Foque em fontes oficiais de saúde (ministérios,
    hospitais, organizações de primeiros socorros). Limite a busca a 5 resultados mais relevantes.
    """,
    description="Agente para buscar informações de primeiros socorros no Google.",
    tools=[google_search] # Este agente utiliza a ferramenta de busca do Google.
)

In [79]:
# Agente 4: Agente de Avaliação de Emergência
agente_avaliacao_emergencia = Agent(
    name="agente_avaliacao_emergencia",
    model="gemini-1.5-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Analise a **descrição completa dos sintomas** e os **resultados da busca por primeiros socorros**
    para determinar a gravidade da situação.
    Se a situação for de emergência (e.g., suspeita de infarto, dificuldade respiratória grave,
    perda de consciência), **INSTRUIA O USUÁRIO A LIGAR IMEDIATAMENTE PARA O SERVIÇO DE EMERGÊNCIA (192 - SAMU no Brasil).**
    Caso contrário, oriente o usuário a procurar atendimento médico em um hospital ou clínica.
    Seja extremamente claro e direto na recomendação de emergência. Não faça mais perguntas ao paciente.
    """,
    description="Agente para avaliar a gravidade da situação e recomendar a ação apropriada."
)

In [80]:
# Agente 5: Agente de Planejamento de Ações
agente_planejamento_acoes = Agent(
    name="agente_planejamento_acoes",
    model="gemini-1.5-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Analise os resultados da busca do Google fornecidos e crie um plano conciso e lógico de
    ações de primeiros socorros a serem recomendadas ao usuário. O plano deve ser claro,
    objetivo e priorizar as ações mais urgentes. Inclua, se necessário, orientações sobre
    quando procurar ajuda médica profissional e como proceder até lá. Escreva da forma mais direta e enxuta possível.
    """,
    description="Agente para planejar as ações de primeiros socorros com base em resultados de busca."
    # Não possui ferramentas, pois analisa dados já fornecidos.
)

In [81]:
# Agente 6: Agente de Raciocínio de Localização de Ajuda Médica
agente_localizacao_ajuda_medica = Agent(
    name="agente_localizacao_ajuda_medica",
    model="gemini-1.5-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Com base na localização fornecida e no planejamento criado, você pode raciocinar sobre a necessidade de hospitais e médicos.
    No entanto, a busca real e a geração de rotas serão feitas por funções auxiliares, que te entregarão os dados.
    Apenas indique que a busca por locais próximos está sendo realizada.
    """,
    description="Agente para raciocinar sobre a necessidade de localizar ajuda médica."
)

In [82]:
# Agente 7: Agente de Raciocínio de Rota e Navegação
agente_rota_navegacao = Agent(
    name="agente_rota_navegacao",
    model="gemini-1.5-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Com base nos hospitais encontrados e na sua localização, você pode raciocinar sobre a necessidade de rotas.
    No entanto, a geração de links de rotas será feita por funções auxiliares, que te entregarão os dados.
    Apenas indique que as rotas estão sendo geradas.
    """,
    description="Agente para raciocinar sobre rotas para hospitais."
)

In [85]:
# Agente 8: Agente de Orientação Detalhada Final
agente_orientacao_detalhada = Agent(
    name="agente_orientacao_detalhada",
    model="gemini-1.5-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Com base na **descrição completa dos sintomas, resultados de busca, plano de ações e avaliação de emergência** fornecidos,
    gere um guia claro e acionável com instruções gerais e fáceis de seguir para o usuário.

    **Formate a saída utilizando títulos e subtítulos em negrito, e use tópicos e subtópicos com bullet points (usando '*' ou '-') para melhor clareza e legibilidade. Garanta que o texto dentro dos bullet points não tenha formatação em negrito extra, a menos que seja um termo específico que mereça destaque.**
    Organize as informações em seções concisas, seguindo esta estrutura:

    * **1. Avaliação da Situação:** (Resuma a gravidade da situação com base na avaliação de emergência.)
    * **2. Ações de Primeiros Socorros Imediatas:** (Apresente o plano de ações de forma clara e numerada ou com bullet points, com as ações mais urgentes primeiro.)
        * (Ex: Posicionar a pessoa confortavelmente.)
        * (Ex: Aplicar compressa fria, se indicado.)
    * **3. Quando Procurar Ajuda Médica/Serviços de Emergência:** (Detalhe os critérios para buscar socorro profissional ou ligar para 192/SAMU, se aplicável.)
    * **4. Orientações Gerais e Próximos Passos:** (Inclua informações adicionais relevantes. **Use a ferramenta Google Search para encontrar e incluir links para vídeos do YouTube que demonstrem, de forma clara e segura, técnicas de primeiros socorros, procedimentos ou exames práticos relacionados à condição. Priorize links de fontes de saúde confiáveis (ex: hospitais, organizações médicas, canais educativos).** Reafirme a importância de monitoramento e busca de ajuda se a situação piorar.)

    **É fundamental mencionar a importância de procurar ajuda médica profissional imediatamente se a situação for grave, e orientar sobre como proceder até a chegada do socorro ou deslocamento para um local de atendimento.**
    Seja conciso, mas completo, fornecendo os próximos passos mais importantes em um formato de lista.
    """,
    description="Agente para gerar orientações detalhadas finais de primeiros socorros, compilando todas as informações.",
    tools=[google_search] # Este agente utiliza a ferramenta de busca do Google.
)


In [89]:
# --- Funções Auxiliares para a API do Google Maps (Assíncronas) ---
# Estas funções interagem com a API do Google Places para buscar locais próximos.
# São assíncronas para melhor performance em operações de rede.

async def buscar_hospitais_proximos(latitude: float, longitude: float, raio: int = DEFAULT_SEARCH_RADIUS_KM) -> List[Dict]:
    """
    Busca hospitais próximos a uma dada localização usando a API Google Places Nearby Search.
    Limita os resultados a no máximo 4 hospitais para concisão. Inclui tratamento de erros
    para requisições de rede e respostas da API.

    Args:
        latitude (float): Latitude da localização do usuário.
        longitude (float): Longitude da localização do usuário.
        raio (int): Raio de busca em quilômetros. Será convertido para metros na URL da API.

    Returns:
        List[Dict]: Uma lista de dicionários, cada um representando um hospital com nome, endereço,
                    localização (latitude/longitude) e avaliação, se disponível.
                    Retorna uma lista vazia em caso de erro ou nenhum resultado.
    """
    # Constrói a URL para a API Nearby Search do Google Places para hospitais.
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={raio * 1000}&type=hospital&key={MAPS_API_KEY}&language=pt"

    # Utiliza aiohttp.ClientSession como um gerenciador de contexto assíncrono para garantir
    # que a sessão seja fechada corretamente, mesmo em caso de erros.
    async with aiohttp.ClientSession() as session:
        try:
            # Faz a requisição GET assíncrona para a URL.
            async with session.get(url) as response:
                # Lança uma exceção HTTPError para respostas HTTP de status 4xx/5xx.
                response.raise_for_status()
                # Parseia a resposta JSON.
                data = await response.json()

                if data["status"] == "OK":
                    hospitais = []
                    # Itera sobre os resultados e extrai as informações relevantes dos hospitais.
                    for result in data["results"]:
                        hospitais.append({
                            "nome": result.get("name"),
                            "endereco": result.get("vicinity"),
                            "localizacao": result.get("geometry", {}).get("location"),
                            "rating": result.get("rating")
                        })
                    return hospitais[:4] # Retorna no máximo os 4 primeiros hospitais encontrados.
                elif data["status"] == "ZERO_RESULTS":
                    print("Não foram encontrados hospitais na área especificada.")
                    return []
                else:
                    # Captura outros status de erro da API do Google Places.
                    error_message = data.get('error_message', 'Mensagem de erro não disponível.')
                    print(f"Erro ao buscar hospitais: {data['status']} - {error_message}")
                    return []
        except asyncio.TimeoutError:
            # Exceção específica para timeout da requisição.
            print(f"Tempo limite excedido ao buscar hospitais. Tente novamente ou reduza o raio de busca.")
            return []
        except aiohttp.ClientError as e:
            # Exceção para erros gerais de requisição HTTP (conexão, DNS, etc.).
            print(f"Erro de requisição HTTP ao buscar hospitais: {e}")
            return []
        except Exception as e:
            # Captura quaisquer outras exceções inesperadas.
            print(f"Erro inesperado ao buscar hospitais: {e}")
            return []

async def buscar_medicos_proximos(latitude: float, longitude: float, raio: int = DEFAULT_SEARCH_RADIUS_KM) -> List[Dict]:
    """
    Busca médicos/clínicas próximos a uma dada localização usando a API Google Places Nearby Search.
    Limita os resultados a no máximo 4 médicos/clínicas para concisão. Inclui tratamento de erros
    para requisições de rede e respostas da API.

    Args:
        latitude (float): Latitude da localização do usuário.
        longitude (float): Longitude da localização do usuário.
        raio (int): Raio de busca em quilômetros. Será convertido para metros na URL da API.

    Returns:
        List[Dict]: Uma lista de dicionários, cada um representando um médico/clínica com nome, endereço,
                    localização (latitude/longitude) e avaliação, se disponível.
                    Retorna uma lista vazia em caso de erro ou nenhum resultado.
    """
    # Constrói a URL para a API Nearby Search do Google Places para médicos.
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={raio * 1000}&type=doctor&key={MAPS_API_KEY}&language=pt"

    # Utiliza aiohttp.ClientSession como um gerenciador de contexto assíncrono.
    async with aiohttp.ClientSession() as session:
        try:
            # Faz a requisição GET assíncrona.
            async with session.get(url) as response:
                response.raise_for_status() # Lança uma exceção HTTPError.
                data = await response.json()

                if data["status"] == "OK":
                    medicos = []
                    # Itera sobre os resultados e extrai as informações relevantes dos médicos/clínicas.
                    for result in data["results"]:
                        medicos.append({
                            "nome": result.get("name"),
                            "endereco": result.get("vicinity"),
                            "localizacao": result.get("geometry", {}).get("location"),
                            "rating": result.get("rating")
                        })
                    return medicos[:4] # Retorna no máximo os 4 primeiros médicos/clínicas encontrados.
                elif data["status"] == "ZERO_RESULTS":
                    print("Não foram encontrados médicos na área especificada.")
                    return []
                else:
                    error_message = data.get('error_message', 'Mensagem de erro não disponível.')
                    print(f"Erro ao buscar médicos: {data['status']} - {error_message}")
                    return []
        except asyncio.TimeoutError:
            print(f"Tempo limite excedido ao buscar médicos. Tente novamente ou reduza o raio de busca.")
            return []
        except aiohttp.ClientError as e:
            print(f"Erro de requisição HTTP ao buscar médicos: {e}")
            return []
        except Exception as e:
            print(f"Erro inesperado ao buscar médicos: {e}")
            return []

async def gerar_rotas_Maps(origem_lat: float, origem_long: float, destinos: List[Dict]) -> List[str]:
    """
    Gera URLs diretas para rotas no Google Maps para navegação.
    As URLs são abertas em uma nova aba ao serem clicadas. Limita a 4 links de rota.

    Args:
        origem_lat (float): Latitude do ponto de origem (usuário).
        origem_long (float): Longitude do ponto de origem (usuário).
        destinos (List[Dict]): Lista de dicionários de destinos (hospitais/médicos)
                                contendo suas informações de localização.

    Returns:
        List[str]: Uma lista de URLs do Google Maps para as rotas geradas.
    """
    rotas = []
    # Itera sobre os primeiros 4 destinos para gerar as URLs das rotas.
    for destino in destinos[:4]:
        # Verifica se as informações de localização necessárias estão presentes no destino.
        if "localizacao" in destino and "lat" in destino["localizacao"] and "lng" in destino["localizacao"]:
            destino_lat = destino["localizacao"]["lat"]
            destino_long = destino["localizacao"]["lng"]

            # Constrói a URL do Google Maps para obter direções.
            # 'origin' define o ponto de partida e 'destination' o ponto de chegada.
            # 'travelmode=driving' especifica o modo de viagem (carro).
            url = f"https://www.google.com/maps/dir/{origem_lat},{origem_long}/{destino_lat},{destino_long}/data=!4m2!4m1!3e0?entry=ttu"
            rotas.append(url)
        else:
            print(f"Localização inválida para destino: {destino.get('nome', 'N/A')}. Rota não gerada.")
    return rotas # Retorna a lista de URLs de rotas.

# --- Função Orquestrador Principal ---
async def orquestrador_primeiros_socorros(mensagem_usuario: str, latitude_usuario: float, longitude_usuario: float):
    """
    Orquestra a interação entre os diversos agentes de IA e funções auxiliares,
    guiando o usuário através de um processo estruturado de assistência de primeiros socorros.

    Args:
        mensagem_usuario (str): A descrição inicial dos sintomas fornecida pelo usuário.
        latitude_usuario (float): Latitude da localização atual do usuário.
        longitude_usuario (float): Longitude da localização atual do usuário.
    """
    print("\n" + "="*70)
    print("⚕️ Sistema de Assistência de Primeiros Socorros Ativado ⚕️".center(70))
    print("Siga as orientações para obter a melhor assistência.".center(70))
    print("="*70 + "\n")

    # Passo 1: Triagem Inicial dos Sintomas
    # O agente de triagem coleta a descrição inicial dos sintomas do usuário.
    print("\n--- 🗣️ AGENTE DE TRIAGEM: Analisando seus sintomas iniciais... ---\n")
    resposta_triagem = executar_agente(agente_triagem, mensagem_usuario)
    display(to_markdown(f"**Agente de Triagem:** {resposta_triagem}"))
    print("--------------------------------------------------------------")

    # Passo 2: Coleta de Sintomas Adicionais
    # O sistema solicita informações adicionais para ter um quadro de saúde mais completo.
    print("❓ Você sente mais alguma dor ou algo diferente?")
    # O prompt do input é mantido curto para minimizar problemas de layout.
    mensagem_adicional_usuario = input("   (Digite 'não' se não houver): ")

    # O agente de consolidação de sintomas combina as informações iniciais e adicionais.
    print("\n--- 🗣️ AGENTE DE SINTOMAS ADICIONAIS: Consolidando as informações... ---\n")
    sintomas_consolidados = executar_agente(agente_coleta_sintomas_adicionais,
                                            f"Resumo dos sintomas iniciais:\n{resposta_triagem}\n\nSintomas adicionais informados pelo usuário:\n{mensagem_adicional_usuario}")
    display(to_markdown(f"**Sintomas Consolidados:**\n{sintomas_consolidados}"))
    print("--------------------------------------------------------------")

    # Passo 3: Busca por Informações de Primeiros Socorros (Saída Suprimida)
    # O agente de busca utiliza a ferramenta Google Search para encontrar orientações confiáveis.
    # Os resultados não são exibidos diretamente, mas são passados para os próximos agentes.
    print("\n--- 🔍 AGENTE DE BUSCA: Pesquisando as melhores práticas de primeiros socorros... ---\n")
    # A linha abaixo executa a busca, mas removemos o 'display()' para não printar os resultados.
    resultados_busca = executar_agente(agente_busca_primeiros_socorros, sintomas_consolidados)
    print("A busca de informações de primeiros socorros foi concluída internamente.")
    print("--------------------------------------------------------------")

    # Passo 4: Avaliação da Gravidade da Emergência
    # AGORA: O agente de avaliação determina a urgência da situação com base apenas nos sintomas e na busca.
    print("\n--- 🚨 AGENTE DE AVALIAÇÃO DE EMERGÊNCIA: Analisando a gravidade da situação... ---\n")
    # A instrução do agente de avaliação foi adaptada para esta nova ordem.
    recomendacao_emergencia = executar_agente(agente_avaliacao_emergencia,
                                              f"Descrição dos sintomas: {sintomas_consolidados}\nResultados da busca: {resultados_busca}")
    display(to_markdown(f"**Avaliação e Recomendação de Emergência:**\n{recomendacao_emergencia}"))
    print("--------------------------------------------------------------")

    # Passo 5: Planejamento das Ações de Primeiros Socorros
    # AGORA: Com base nos resultados da busca, o agente de planejamento formula um plano de ação inicial.
    print("\n--- 📝 AGENTE DE PLANEJAMENTO: Elaborando um plano de ação... ---\n")
    plano_acoes = executar_agente(agente_planejamento_acoes, resultados_busca)
    display(to_markdown(f"**Plano de Ações Sugerido:**\n{plano_acoes}"))
    print("--------------------------------------------------------------")

    # Passo 6: Busca de Locais de Ajuda Médica Próximos
    # Aciona as funções de Maps para encontrar hospitais e médicos.
    print("\n--- 🏥 AGENTE DE LOCALIZAÇÃO: Buscando hospitais e médicos próximos a você... ---\n")
    # O agente apenas sinaliza o que está acontecendo, a busca é feita pelas funções auxiliares.
    executar_agente(agente_localizacao_ajuda_medica, "Coletando localização do usuário e buscando locais de ajuda.")

    hospitais = await buscar_hospitais_proximos(latitude_usuario, longitude_usuario)
    medicos = await buscar_medicos_proximos(latitude_usuario, longitude_usuario)

    print("\n**Hospitais Próximos (Top 4):**")
    if hospitais:
        for hospital in hospitais:
            display(to_markdown(f"- **{hospital.get('nome')}** ({hospital.get('endereco')}, Avaliação: {hospital.get('rating', 'N/A')})"))
    else:
        display(to_markdown("Não foram encontrados hospitais próximos."))

    print("\n**Médicos/Clínicas Próximos (Top 4):**")
    if medicos:
        for medico in medicos:
            display(to_markdown(f"- **{medico.get('nome')}** ({medico.get('endereco')}, Avaliação: {medico.get('rating', 'N/A')})"))
    else:
        display(to_markdown("Não foram encontrados médicos/clínicas próximos."))
    print("--------------------------------------------------------------")

    # Passo 7: Geração de Rotas para Hospitais
    # Gera links diretos para navegação no Google Maps.
    print("\n--- 🗺️ AGENTE DE NAVEGAÇÃO: Gerando rotas para os locais de ajuda... ---\n")
    # O agente apenas sinaliza o que está acontecendo, a geração é feita pela função auxiliar.
    executar_agente(agente_rota_navegacao, "Gerando links de rotas para os hospitais encontrados.")

    rotas_hospitais = await gerar_rotas_Maps(latitude_usuario, longitude_usuario, hospitais)
    if rotas_hospitais:
        # Exibe os links de rota de forma clicável.
        for i, rota in enumerate(rotas_hospitais):
            hospital_nome = hospitais[i].get('nome', 'Destino') if i < len(hospitais) else 'Destino'
            display(to_markdown(f"Rota para **{hospital_nome}**:"))
            display(HTML(f'<a href="{rota}" target="_blank">Clique aqui para a rota no Google Maps</a>'))
    else:
        display(to_markdown("Não foi possível gerar rotas para os hospitais encontrados."))
    print("--------------------------------------------------------------")

    # Passo 8: Orientação Detalhada Final (Síntese Completa)
    # Este agente compila todas as informações (sintomas, busca, plano, emergência, locais)
    # para gerar a orientação final e completa para o usuário.
    print("\n--- ✨ AGENTE DE ORIENTAÇÃO DETALHADA: Compilando seu guia de ação final... ---\n")
    # Constrói um contexto rico para o agente de orientação detalhada.
    contexto_completo_para_orientacao = (
        f"Sintomas consolidados: {sintomas_consolidados}\n\n"
        f"Resultados da busca por primeiros socorros: {resultados_busca}\n\n"
        f"Plano de ações recomendado: {plano_acoes}\n\n"
        f"Avaliação de emergência: {recomendacao_emergencia}\n\n"
        "Locais de ajuda médica encontrados (hospitais e médicos foram listados com endereços e avaliações): "
        f"{'Hospitais: ' + ', '.join([h['nome'] for h in hospitais]) if hospitais else 'Nenhum hospital encontrado'}. "
        f"{'Médicos/Clínicas: ' + ', '.join([m['nome'] for m in medicos]) if medicos else 'Nenhum médico/clínica encontrado'}."
    )
    orientacao_detalhada_final = executar_agente(agente_orientacao_detalhada, contexto_completo_para_orientacao)

    # --- RESUMO FINAL DA ASSISTÊNCIA AO USUÁRIO ---
    # Esta seção apresenta um resumo consolidado e focado nas ações imediatas para o usuário.
    print("\n" + "*"*70)
    print("🤖 RESUMO FINAL DA ASSISTÊNCIA DE PRIMEIROS SOCORROS 🤖".center(70))
    print("*"*70 + "\n")

    resposta_final = f"""
**1. Orientações de Primeiros Socorros:**

{textwrap.indent(orientacao_detalhada_final.strip(), '    ')}

<br>

**2. Recomendação Urgente:**
{textwrap.indent(recomendacao_emergencia.strip(), '    ')}

---
**IMPORTANTE:** Para detalhes sobre Hospitais, Médicos/Clínicas e Rotas, por favor,
**role para cima** e verifique as mensagens anteriores. Os links para o Google Maps estão lá!
"""
    display(Markdown(resposta_final))
    print("\n" + "="*70)
    print("Processo de Assistência Concluído. Fique bem!".center(70))
    print("="*70 + "\n")


# --- Execução Principal do Programa ---
async def main():
    """
    Função principal que inicia o fluxo do sistema de primeiros socorros.
    Ela coleta a descrição inicial do usuário e define uma localização de teste.
    Em uma aplicação real, a localização seria obtida dinamicamente.
    """
    # Solicita a descrição inicial dos sintomas ao usuário.
    mensagem_usuario = input("❓ Olá! Para iniciar, por favor, descreva o que você está sentindo (ex: 'Dor no peito e formigamento no braço esquerdo'): ")

    # Localização fixa para testes (Campinas, SP, Brasil).
    # Em uma aplicação de produção, esta localização seria obtida dinamicamente do usuário.
    latitude_usuario = -22.9056
    longitude_usuario = -47.0608

    # Chama a função orquestradora principal para iniciar a jornada de assistência.
    await orquestrador_primeiros_socorros(mensagem_usuario, latitude_usuario, longitude_usuario)

# Garante que a função 'main' seja executada quando o script é iniciado.
# 'asyncio.run(main())' é o método padrão para rodar funções assíncronas.
if __name__ == "__main__":
    asyncio.run(main())

❓ Olá! Para iniciar, por favor, descreva o que você está sentindo (ex: 'Dor no peito e formigamento no braço esquerdo'): Febre

      ⚕️ Sistema de Assistência de Primeiros Socorros Ativado ⚕️      
         Siga as orientações para obter a melhor assistência.         


--- 🗣️ AGENTE DE TRIAGEM: Analisando seus sintomas iniciais... ---



> **Agente de Triagem:** Você está com febre?  Quanto tempo você está se sentindo assim?  Você está sentindo algum outro sintoma, como dor de cabeça, calafrios, tosse ou dor de garganta? Você sente mais alguma dor ou algo diferente que eu deva saber?

--------------------------------------------------------------
❓ Você sente mais alguma dor ou algo diferente?
   (Digite 'não' se não houver): não

--- 🗣️ AGENTE DE SINTOMAS ADICIONAIS: Consolidando as informações... ---



> **Sintomas Consolidados:**
> Febre, duração desconhecida.  Sem outros sintomas relatados além da febre.

--------------------------------------------------------------

--- 🔍 AGENTE DE BUSCA: Pesquisando as melhores práticas de primeiros socorros... ---

A busca de informações de primeiros socorros foi concluída internamente.
--------------------------------------------------------------

--- 🚨 AGENTE DE AVALIAÇÃO DE EMERGÊNCIA: Analisando a gravidade da situação... ---



...

> Rota para **Hospital Beneficência Portuguesa**:

> Rota para **Hospital Santa Elisa**:

--------------------------------------------------------------

--- ✨ AGENTE DE ORIENTAÇÃO DETALHADA: Compilando seu guia de ação final... ---


**********************************************************************
        🤖 RESUMO FINAL DA ASSISTÊNCIA DE PRIMEIROS SOCORROS 🤖         
**********************************************************************




**1. Orientações de Primeiros Socorros:**

    **1. Avaliação da Situação:**

    A situação é considerada de **potencial gravidade**, pois a febre, sem outros sintomas, pode indicar uma variedade de condições, algumas delas graves. A ausência de informações sobre a duração da febre torna a avaliação ainda mais complexa, demandando uma investigação médica completa.  Não é possível determinar a gravidade sem uma avaliação médica profissional.


    **2. Ações de Primeiros Socorros Imediatas:**

    * **Monitoramento:** Meça a temperatura regularmente (a cada 4 horas, por exemplo) usando um termômetro e anote a hora e a temperatura. Observe a duração da febre.  Anote qualquer outro sintoma que possa surgir, mesmo que pareça insignificante.
    * **Hidratação:** Beba bastante líquido (água, sucos naturais, caldos) para evitar a desidratação.
    * **Conforto:** Use roupas leves e arejadas.  Utilize compressas mornas ou frias na testa, dependendo da sua preferência e conforto.
    * **Alimentação leve:** Consuma alimentos leves e fáceis de digerir, se tiver apetite.  Caso não tenha apetite, não se force a comer. A hidratação é mais importante nesse momento.
    * **Analgesia/Antitérmicos (com cautela):** Paracetamol ou ibuprofeno podem ser usados para reduzir o desconforto da febre, seguindo rigorosamente as instruções da bula e a dosagem recomendada para o seu peso e idade. **Não exceda a dosagem recomendada e não utilize por tempo prolongado sem orientação médica.**


    **3. Quando Procurar Ajuda Médica/Serviços de Emergência:**

    Procure imediatamente ajuda médica se:

    * A febre persistir por mais de 3 dias em adultos ou menos em crianças (a duração varia de acordo com a idade e condições de saúde pré-existentes).
    * A febre estiver acompanhada de outros sintomas, mesmo leves (dor de cabeça, fraqueza, tosse, vômitos, diarreia, dor muscular, manchas na pele, etc.).
    * A febre for alta (acima de 39°C).
    * Houve exposição recente a doenças infecciosas.
    * Houve viagens recentes, principalmente para áreas com risco de doenças infecciosas.
    * Você apresentar sinais de desidratação (boca seca, urina escura, tontura, fraqueza excessiva).
    * Você sentir falta de ar ou dificuldade em respirar.
    * Você tiver convulsões.


    **4. Orientações Gerais e Próximos Passos:**

    * **Evite a automedicação:** Não utilize antibióticos ou outros medicamentos sem orientação médica, pois isso pode mascarar os sintomas e dificultar o diagnóstico.
    * **Repouso:** Descanse bastante para ajudar o seu corpo a combater a infecção, se houver.
    * **Higiene:** Lave bem as mãos com frequência para evitar a propagação de infecções.
    * **Informação ao Médico:** Ao consultar o médico, forneça todas as informações relevantes: duração da febre, medicamentos utilizados (incluindo remédios sem prescrição médica), viagens recentes, histórico médico, e quaisquer outros sintomas, por mais leves que sejam.
    * **Monitoramento contínuo:** Continue monitorando sua temperatura e observando outros sintomas. Se a situação piorar ou surgirem novos sintomas preocupantes, procure ajuda médica imediatamente.  Ligue para o serviço de emergência (192/SAMU) se necessário.


    **Observação:**  Este guia fornece informações gerais e não substitui a consulta médica.  A busca por ajuda profissional é fundamental para um diagnóstico preciso e o tratamento adequado.  A informação apresentada aqui é válida para hoje, 18 de Maio de 2025, e pode sofrer alterações.

<br>

**2. Recomendação Urgente:**
    Procure atendimento médico imediatamente em um hospital ou clínica.  A febre, sem informações sobre a duração e a ausência de outros sintomas, requer avaliação médica para determinar a causa. Não se automedique.

---
**IMPORTANTE:** Para detalhes sobre Hospitais, Médicos/Clínicas e Rotas, por favor,
**role para cima** e verifique as mensagens anteriores. Os links para o Google Maps estão lá!



            Processo de Assistência Concluído. Fique bem!             

